In [1]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, multilabel_confusion_matrix 
from sklearn.model_selection import train_test_split
from ModelHandler import ModelHandler
import pickle
import h5py
from sklearn.metrics import f1_score, precision_score, recall_score     

folderPath = "Weights/Final/cenario13/"
folderDataPath = "Weights/Final/cenario13/"
signalBaseLength = 12800
trainSize = 0.8
ngrids = 5

FOLD = True

dict_data = pickle.load(open(folderDataPath + "sorted_aug_data_" + str(ngrids) + "_" + str(signalBaseLength) + ".p", "rb")) # Load data
x_train = dict_data["x_train"]
x_test = dict_data["x_test"]
y_train = dict_data["y_train"]
y_test = dict_data["y_test"]

datasetPath = "Synthetic_Full_iHall.hdf5"

all_labels = []

arq = h5py.File(datasetPath, "r")
loads_list = ["1", "2", "3", "8"]
for load_qtd in loads_list:
    labels = arq[load_qtd]["labels"]  
    events = arq[load_qtd]["events"]    
    for waveform_labels, event in zip(labels, events):
        event_index = np.argwhere(event != 0)
        # events = event[event_index]
        for label, ev in zip(waveform_labels, event_index):
            all_labels.append(event[ev][0] * (label + 1))

copia_all_labels = all_labels.copy()
for i in range(int((len(y_test["classification"]) + len(y_train["classification"]))/len(copia_all_labels)) - 1):
    for label in copia_all_labels:
        all_labels.append(label)

labels_train, labels_test = train_test_split(all_labels, train_size=int(trainSize * len(all_labels)), random_state = 42)

In [4]:
if FOLD == False:
    threshold = 0.5
    final_prediction = []
    final_prediction_with_detection = []
    final_groundTruth = []

    bestModel = ModelHandler.loadModel(folderPath + "best_model.h5", type_weights=None) # Load model
    for xi, yclass, ytype in zip(x_test, y_test["classification"], y_test["type"]):
        pred = bestModel.predict(np.expand_dims(xi, axis=0))
        prediction = np.max(pred[2][0],axis=0) > threshold
        groundTruth = np.max(yclass,axis=0) > threshold

        det = np.array([np.argmax(i) for i in ytype])
        prediction_with_detection = np.max(pred[2][0],axis=0) > 2 # Gambiarra
        if (det != 2).any():
            prediction_with_detection = np.max(pred[2][0],axis=0) > threshold

        final_prediction.append(prediction)
        final_groundTruth.append(groundTruth)
        final_prediction_with_detection.append(prediction_with_detection)

In [5]:
if FOLD == False:
    correct_on = np.zeros((26,1))
    total_on = np.zeros((26,1))
    correct_off = np.zeros((26,1))
    total_off = np.zeros((26,1))

    for ytrue, ypred in zip(labels_test, final_prediction):
        if ytrue > 0:
            ytrue = ytrue - 1
            total_on[ytrue] += 1
            if ypred[ytrue] == 1:
                correct_on[ytrue] += 1
        elif ytrue < 0:
            ytrue = -1 * ytrue - 1 # Multiplica por -1 para voltar a ser positivo e subtrai 1 para voltar a começar do índice 0
            total_off[ytrue] += 1
            if ypred[ytrue] == 1:
                correct_off[ytrue] += 1

    acc_on = np.average(correct_on/total_on) % 100
    acc_off = np.average(correct_off/total_off) % 100
    acc = np.average((correct_on + correct_off)/(total_on + total_off)) % 100
    print(f"Acc total: {acc}, Acc on: {acc_on}, Acc off: {acc_off}")

In [2]:
if FOLD == True:
    X_all = np.vstack((x_train, x_test))
    ydet_all = np.vstack((y_train["detection"], y_test["detection"]))
    ytype_all = np.vstack((y_train["type"], y_test["type"]))
    yclass_all = np.vstack((y_train["classification"], y_test["classification"]))
    all_labels = np.vstack((labels_train, labels_test))

    final_acc_on, final_acc_off, final_acc = [], [], []
    y = {}
    for fold in range(1, 11):
        foldFolderPath = folderPath + str(fold) + "/"
        
        train_index = np.load(foldFolderPath + "train_index.npy")
        test_index = np.load(foldFolderPath + "test_index.npy")

        bestModel = ModelHandler.loadModel(foldFolderPath + "best_model.h5", type_weights=None) # Load model

        x_train = X_all[train_index]
        x_test = X_all[test_index]
        y_train["detection"] = ydet_all[train_index]
        y_test["detection"] = ydet_all[test_index]
        y_train["type"] = ytype_all[train_index]
        y_test["type"] = ytype_all[test_index]
        y_train["classification"] = yclass_all[train_index]
        y_test["classification"] = yclass_all[test_index]

        threshold = 0.5
        final_prediction = []
        final_prediction_with_detection = []
        final_groundTruth = []
        for xi, yclass, ytype in zip(x_test, y_test["classification"], y_test["type"]):
            pred = bestModel.predict(np.expand_dims(xi, axis=0))
            prediction = np.max(pred[2][0],axis=0)
            groundTruth = np.max(yclass,axis=0)

            det = np.array([np.argmax(i) for i in ytype])
            prediction_with_detection = np.max(pred[2][0],axis=0) > 2 # Gambiarra
            if (det != 2).any():
                prediction_with_detection = np.max(pred[2][0],axis=0)

            final_prediction.append(prediction)
            final_groundTruth.append(groundTruth) 
            final_prediction_with_detection.append(prediction_with_detection)

            del xi, yclass, ytype

        y[fold] = {}
        y[fold]["true"] = final_groundTruth.copy()
        y[fold]["pred"] = final_prediction.copy()
        y[fold]["pred_with_detection"] = final_prediction_with_detection.copy()

        print(f"Predicted fold {fold}")

Predicted fold 1
Predicted fold 2
Predicted fold 3
Predicted fold 4
Predicted fold 5
Predicted fold 6
Predicted fold 7
Predicted fold 8
Predicted fold 9
Predicted fold 10


In [7]:
if FOLD == True:
    all_labels = np.vstack((labels_train, labels_test))

    # acumulative_acc_on, acumulative_acc_off, acumulative_total_on, acumulative_total_off = [], [], [], []

    final_acc_on, final_acc_off, final_acc = [], [], []
    for fold in range(1, 11):
        correct_on = np.zeros((26,1))
        total_on = np.zeros((26,1))
        correct_off = np.zeros((26,1))
        total_off = np.zeros((26,1))

        train_index = np.load(folderPath + str(fold) + "/train_index.npy")
        test_index = np.load(folderPath + str(fold) + "/test_index.npy")

        labels_train = all_labels[train_index]
        labels_test = all_labels[test_index]

        for ytrue, ypred in zip(labels_test, y[fold]["pred"]):
            if ytrue > 0:
                ytrue = ytrue - 1
                total_on[ytrue] += 1
                if ypred[ytrue] == 1:
                    correct_on[ytrue] += 1
            elif ytrue < 0:
                ytrue = -1 * ytrue - 1 # Multiplica por -1 para voltar a ser positivo e subtrai 1 para voltar a começar do índice 0
                total_off[ytrue] += 1
                if ypred[ytrue] == 1:
                    correct_off[ytrue] += 1
            else:
                print(ytrue)

        acc_on = np.average(np.nan_to_num(correct_on/total_on, nan=0)) * 100
        acc_off = np.average(np.nan_to_num(correct_off/total_off, nan=0)) * 100
        acc = np.average(np.nan_to_num((correct_on + correct_off)/(total_on + total_off), nan=0)) * 100
        print(f"Fold: {fold}, Acc total: {acc}, Acc on: {acc_on}, Acc off: {acc_off}")
        print(np.sum(total_on), np.sum(total_off))

        final_acc_on.append(acc_on)
        final_acc_off.append(acc_off)
        final_acc.append(acc)

        # acumulative_acc_on.append(correct_on)
        # acumulative_acc_off.append(correct_off)
        # acumulative_total_on.append(total_on)
        # acumulative_total_off.append(total_off)

    print(f"Average - Acc total: {np.average(final_acc)}, Acc on: {np.average(final_acc_on)}, Acc off: {np.average(final_acc_off)}")

    # acumulative_acc_on = np.sum(acumulative_acc_on, axis=0)
    # acumulative_acc_off = np.sum(acumulative_acc_off, axis=0)
    # acumulative_total_on = np.sum(acumulative_total_on, axis=0)
    # acumulative_total_off = np.sum(acumulative_total_off, axis=0)

    # print(f"Average - Acc total: {np.average((acumulative_acc_on + acumulative_acc_off)/(acumulative_total_on + acumulative_total_off))}, Acc on: {np.average(acumulative_acc_on/acumulative_total_on)}, Acc off: {np.average(acumulative_acc_off/acumulative_total_off)}")

Fold: 1, Acc total: 96.19179349548342, Acc on: 92.54188709381307, Acc off: 97.5274383166575
1303.0 1222.0
Fold: 2, Acc total: 95.65102289155053, Acc on: 94.03519850479867, Acc off: 98.40275959687536
1273.0 1252.0
Fold: 3, Acc total: 95.3075428706147, Acc on: 93.1430313165447, Acc off: 97.41903864963594
1246.0 1279.0
Fold: 4, Acc total: 96.61971681945396, Acc on: 93.20210057918959, Acc off: 97.74236645839497
1238.0 1287.0
Fold: 5, Acc total: 96.85212387826562, Acc on: 96.51611396812046, Acc off: 97.19659215930764
1262.0 1263.0
Fold: 6, Acc total: 96.69528756713763, Acc on: 95.8988807370545, Acc off: 97.48798114609592
1276.0 1249.0
Fold: 7, Acc total: 95.70459625217663, Acc on: 94.47511607902067, Acc off: 96.71021932396536
1265.0 1260.0
Fold: 8, Acc total: 94.37035616309531, Acc on: 93.28365350513383, Acc off: 96.54961562950955
1286.0 1239.0
Fold: 9, Acc total: 97.2599338441672, Acc on: 96.85725475390774, Acc off: 98.03606136041445
1240.0 1284.0
Fold: 10, Acc total: 95.2306783082388, Acc

### Cálculo da acurácia 

Acurácia a definição e considerando cada saída da rede de classificação como um classificador binário

$$
\begin{gather*}
Acc_i = \frac{TP + TN}{TP + TN + FP + FN} \\ \\
Acc = \frac{1}{N} \sum_{i = 1}^{N} Acc_i
\end{gather*}
$$

- Acc_i: Acurácia para a carga i

In [8]:
threshold = 0.5

all_labels = np.vstack((labels_train, labels_test))

acc_on, acc_off, acc_total = [], [], []
for fold in range(1, 11):
    correct_on = np.zeros((26,1))
    total_on = np.zeros((26,1))
    correct_off = np.zeros((26,1))
    total_off = np.zeros((26,1))

    train_index = np.load(folderPath + str(fold) + "/train_index.npy")
    test_index = np.load(folderPath + str(fold) + "/test_index.npy")

    labels_train = all_labels[train_index]
    labels_test = all_labels[test_index]

    for label, ytrue, ypred in zip(labels_test, y[fold]["true"], y[fold]["pred"]):
        if label > 0:
            # correct_on[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
            # total_on[ytrue > threshold] += 1
            # total_on[np.bitwise_or(ytrue > threshold, ypred > threshold)] += 1
            correct_on[(ytrue > threshold) == (ypred > threshold)] += 1
            total_on += 1
        elif label < 0:
            # correct_off[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
            # total_off[ytrue > threshold] += 1
            # total_off[np.bitwise_or(ytrue > threshold, ypred > threshold)] += 1
            correct_off[(ytrue > threshold) == (ypred > threshold)] += 1
            total_off += 1
        
    acc_on.append(np.average(np.nan_to_num(correct_on/total_on)))
    acc_off.append(np.average(np.nan_to_num(correct_off/total_off)))
    acc_total.append(np.average(np.nan_to_num((correct_on + correct_off)/(total_on + total_off))))

    print(f"Fold {fold}, Acc on: {acc_on[-1]}, Acc off: {acc_off[-1]}, Acc total: {acc_total[-1]}")

print(f"Total, Acc on: {np.average(acc_on)}, Acc off: {np.average(acc_off)}, Acc total: {np.average(acc_total)}")

Fold 1, Acc on: 0.9949524765334435, Acc off: 0.9965693063074406, Acc total: 0.9957349581111957
Fold 2, Acc on: 0.9961326968396882, Acc off: 0.997081592528877, Acc total: 0.9966031987814167
Fold 3, Acc on: 0.9939190023459686, Acc off: 0.9953990497383773, Acc total: 0.9946686976389947
Fold 4, Acc on: 0.9975767366720517, Acc off: 0.997698882314267, Acc total: 0.9976389946686977
Fold 5, Acc on: 0.9960685115201756, Acc off: 0.9969852000730861, Acc total: 0.9965270373191164
Fold 6, Acc on: 0.9967446346756692, Acc off: 0.9971669643407033, Acc total: 0.996953541507997
Fold 7, Acc on: 0.99738522347218, Acc off: 0.9973443223443224, Acc total: 0.9973648134044174
Fold 8, Acc on: 0.9961418830003589, Acc off: 0.9971440988390143, Acc total: 0.9966336633663366
Fold 9, Acc on: 0.9950682382133995, Acc off: 0.9945482866043615, Acc total: 0.9948037303425578
Fold 10, Acc on: 0.9955154157583309, Acc off: 0.9976427761532494, Acc total: 0.996601852980617
Total, Acc on: 0.9959504819031265, Acc off: 0.996758047

In [3]:
threshold = 0.5

all_labels = np.vstack((labels_train, labels_test))

acc_on, acc_off, acc_total = [], [], []
for fold in range(1, 11):
    correct_on = np.zeros((26,1))
    total_on = np.zeros((26,1))
    correct_off = np.zeros((26,1))
    total_off = np.zeros((26,1))

    train_index = np.load(folderPath + str(fold) + "/train_index.npy")
    test_index = np.load(folderPath + str(fold) + "/test_index.npy")

    labels_train = all_labels[train_index]
    labels_test = all_labels[test_index]

    for label, ytrue, ypred in zip(labels_test, y[fold]["true"], y[fold]["pred"]):
        if label > 0:
            correct_on[(ytrue > threshold) == (ypred > threshold)] += 1
            total_on[ytrue > threshold] += 1
        elif label < 0:
            correct_off[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
            total_off[ytrue > threshold] += 1

    # total_on = np.where(total_on == 0, correct_on, total_on)
    # total_off = np.where(total_off == 0, correct_off, total_off)    
    
    acc_on.append(np.average(np.nan_to_num(correct_on/total_on)))
    acc_off.append(np.average(np.nan_to_num(correct_off/total_off)))
    acc_total.append(np.average(np.nan_to_num((correct_on + correct_off)/(total_on + total_off))))

    print(f"Fold {fold}, Acc on: {acc_on[-1]}, Acc off: {acc_off[-1]}, Acc total: {acc_total[-1]}")

print(f"Total, Acc on: {np.average(acc_on)}, Acc off: {np.average(acc_off)}, Acc total: {np.average(acc_total)}")

Fold 1, Acc on: 0.9431281226120443, Acc off: 0.9870077802926042, Acc total: 0.9758275127936348
Fold 2, Acc on: 0.9543069157601172, Acc off: 0.9930085296482326, Acc total: 0.9684629239937494
Fold 3, Acc on: 0.9526084057536229, Acc off: 0.9862392079219131, Acc total: 0.9690908704370735
Fold 4, Acc on: 0.9502807985347455, Acc off: 0.9889651665328318, Acc total: 0.9809545914084766
Fold 5, Acc on: 0.9816589508098907, Acc off: 0.9825139185044682, Acc total: 0.9824282046641915
Fold 6, Acc on: 0.9743425143263184, Acc off: 0.9890447656456492, Acc total: 0.981763816654446
Fold 7, Acc on: 0.9613606732632907, Acc off: 0.9840428812431075, Acc total: 0.9723644291135973
Fold 8, Acc on: 0.9533552515979224, Acc off: 0.985447421355359, Acc total: 0.9623539480934623
Fold 9, Acc on: 0.9844494129346484, Acc off: 0.9892635081876955, Acc total: 0.9864418885754864
Fold 10, Acc on: 0.9313551116986496, Acc off: 0.9921141804938952, Acc total: 0.9696432031934634
Total, Acc on: 0.958684615729125, Acc off: 0.987764